In [ ]:
!pip install tensorflow scikit-learn numpy seaborn matplotlib pillow

In [ ]:
# Facial Emotion Recognition using CNN
# Simple practical demo version - uses local folder dataset (FER-2013 structure)

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import os

# -----------------------------------------------------------
# 1. Dataset Paths  (change this path if needed)
# -----------------------------------------------------------
base_dir = "/content"  # folder that contains 'train' and 'test'
train_dir = os.path.join(base_dir, "/content/train")
test_dir = os.path.join(base_dir, "/content/test")

# -----------------------------------------------------------
# 2. Image Data Generators
# -----------------------------------------------------------
img_size = 48
batch_size = 64

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    color_mode='grayscale',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

# -----------------------------------------------------------
# 3. Build a simple CNN model
# -----------------------------------------------------------
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(48,48,1)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_generator.num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# -----------------------------------------------------------
# 4. Train the model
# -----------------------------------------------------------
print("\n🚀 Training model...")
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10
)

# -----------------------------------------------------------
# 5. Evaluate and print results
# -----------------------------------------------------------
loss, acc = model.evaluate(test_generator)
print(f"\n🎯 Test Accuracy: {acc*100:.2f}%")
print(f"📉 Test Loss: {loss:.4f}")

# -----------------------------------------------------------
# 6. Plot accuracy and loss graphs
# -----------------------------------------------------------
plt.figure(figsize=(10,4))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Loss')
plt.legend()

plt.show()

# -----------------------------------------------------------
# 7. Show 5 sample predictions
# -----------------------------------------------------------
print("\n🧪 Showing 5 random predictions...")
emotion_labels = list(train_generator.class_indices.keys())

x_test, y_test = next(test_generator)
preds = model.predict(x_test)

plt.figure(figsize=(10,2))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(x_test[i].reshape(48,48), cmap='gray')
    pred_label = emotion_labels[np.argmax(preds[i])]
    true_label = emotion_labels[np.argmax(y_test[i])]
    plt.title(f"P:{pred_label}\nT:{true_label}")
    plt.axis('off')
plt.show()

print("\n✅ Demo completed successfully!")
